In [44]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from matplotlib.lines import Line2D


np.random.seed(0)

# Creating the Dataset

In [ ]:
def create_data(points, classes): # Creates a dataset of spiral shaped clusters
    X = np.zeros((points * classes, 2))
    y = np.zeros(points * classes, dtype='uint8')

    for class_number in range(classes):
        ix = range(points * class_number,
                   points * (class_number + 1))
        r = np.linspace(0.0, 1, points)  # radius
        t = np.linspace(class_number * 4,
                        (class_number + 1) * 4,
                        points) + np.random.randn(points) * 0.2
        X[ix] = np.c_[r * np.sin(t * 2.5),
                      r * np.cos(t * 2.5)]
        y[ix] = class_number

    # Remove points where |x| < 0.1 AND |y| < 0.1
    mask = ~((np.abs(X[:, 0]) < 0.1) & (np.abs(X[:, 1]) < 0.1))
    X = X[mask]
    y = y[mask]

    return X, y

X,Y = create_data(300,3)

# Layers

In [46]:
class Neuron_Layer():
    def __init__(self, n_inputs,n_neurons):
        self.weights = 0.1*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1,n_neurons))
        pass

    def forward(self,inputs):
        self.inputs = inputs
        self.output = np.dot(inputs,self.weights)+self.biases
        pass
    
    def backward(self,dvalues):
        # Biases are constants so do not have an effect on dweights
        self.dweights = np.dot(self.inputs.T, dvalues) # f(a,b) = a*b => f`a(a,b) = b
        # # f(a,b) = a + b => f`b(a,b) = 1, accumulate gradient from all samples
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

# Activation Functions

In [47]:
class Activation_ReLU():
    def forward(self,inputs):
        self.output = np.maximum(0,inputs)
        self.inputs = inputs
        
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

In [48]:
class Activation_Softmax():
    def forward(self,inputs):
        exps = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
        normalised = exps / np.sum(exps,axis=1,keepdims=True)
        self.output = normalised
    
    def backward(self,dvalues): 
        # TODO Reread 216-225
        self.dinputs = np.empty_like(dvalues)
        
        for index, (single_output,single_dvalues) in enumerate(zip(self.output,dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - \
                              np.dot(single_output, single_output.T)
                              
            # Calculate sample-wise gradient.
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

# Optimizers

In [49]:
class Optimizer_SGD:
    def __init__(self,learning_rate=1.0, decay=0., momentum=0.):
        self.learning_rate=learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    
    def update_params(self,layer : Neuron_Layer):
        # Momentum helps to not get stuck in a local minimum
        if self.momentum:
            if not hasattr(layer,"weight_momentums"):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            weight_updates = self.momentum * layer.weight_momentums - \
                self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            bias_updates = self.momentum * layer.bias_momentums - \
                self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        layer.weights += weight_updates
        layer.biases += bias_updates
    
    def post_update_params(self):
        self.iterations += 1

# Loss

In [50]:
class Loss:
    def calculate(self,output,y):
        sample_losses = self.forward(output,y)
        data_loss = np.mean(sample_losses)
        return data_loss

In [51]:
class Loss_CategoricalCrossentropy(Loss):
    def forward(self,y_pred,y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)
        
        if len(y_true.shape)==1: # Labels are class indicies
            correct_confidences = y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape)==2: # One hot encoding
            correct_confidences = np.sum(y_pred_clipped*y_true,axis=1)
        
        loss = -np.log(correct_confidences)
        return loss
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape)==1:
            # If it is sparse, convert to one-hot
            y_true = np.eye(labels)[y_true]
        self.dinputs = -y_true/dvalues # Calculate gradient
        self.dinputs = self.dinputs/samples # Normalize gradient

Below is a combined implementation of the softmax activation function and the Categorical Crossentropy loss function, which is more optimal.

In [52]:
class Activation_Softmax_Loss_CategoricalCrossentropy():
    
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
        
    def forward(self,inputs,y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output,y_true)
    
    def backward(self,dvalues, y_true):
        samples = len(dvalues)
        
        if len(y_true.shape)==2: # If one hot, turn it into sparse (discrete)
            y_true = np.argmax(y_true,axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples),y_true] -=1
        self.dinputs = self.dinputs / samples 

# Spiral dataset
In the below cells I will attempt to create and train a neural network on the spiral dataset created in cell 2.

In [53]:
# Defining the structure of the network


layer1 = Neuron_Layer(2,64)
activation1 = Activation_ReLU()

layer2 = Neuron_Layer(64,64) # Hidden layer
activation2 =Activation_ReLU()

layer3 = Neuron_Layer(64,3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_SGD(learning_rate=0.75,decay=0.0001,momentum=0.5)

In [54]:
# Forward pass
layer1.forward(X)
activation1.forward(layer1.output)

layer2.forward(activation1.output)
activation2.forward(layer2.output)

layer3.forward(activation2.output)
loss = loss_activation.forward(layer3.output, Y)

predictions = np.argmax(loss_activation.output,axis=1)
y = Y.copy()
if len(y.shape) == 2:
    y = np.argmax(Y, axis=1)
accuracy = np.mean(predictions==y)
print('loss:', loss)
print('acc:', accuracy)



loss: 1.0985155458304299
acc: 0.2981132075471698




## Backward Pass: Softmax + Categorical Crossentropy

The backward pass through the **softmax** and **categorical crossentropy loss** can be implemented in two different ways.

---

### **Combined Implementation (Recommended)**

This approach fuses the softmax activation and categorical crossentropy loss into a single backward pass.
It is **computationally more efficient** and **numerically stable**.

```python
softmax_loss = Activation_Softmax_Loss_CategoricalCrossentropy()
softmax_loss.backward(activation2.output, Y)
dvalues1 = softmax_loss.dinputs
```

---

### **Separate Implementation (Less Optimal)**

This approach computes the backward pass for the loss and activation **independently**, resulting in extra computation.

```python
activation = Activation_Softmax()
activation.output = activation2.output

loss = Loss_CategoricalCrossentropy()
loss.backward(activation2.output, Y)

activation.backward(loss.dinputs)
dvalues2 = activation.dinputs
```



In [55]:
loss_activation.backward(loss_activation.output, Y)
layer3.backward(loss_activation.dinputs)
activation2.backward(layer3.dinputs)
layer2.backward(activation2.dinputs)
activation1.backward(layer2.dinputs)
layer1.backward(activation1.dinputs)


print(layer1.dweights)
print(layer1.dbiases)
print(layer2.dweights)
print(layer2.dbiases)
print(layer3.dweights)
print(layer3.dbiases)


optimizer.update_params(layer1)
optimizer.update_params(layer2)
optimizer.update_params(layer3)


[[ 1.86233077e-03  4.47138528e-04  8.76981730e-04 -8.27468927e-04
   7.53620084e-04 -9.83849692e-04 -1.34485427e-03 -1.62220298e-04
  -5.21077544e-04  7.45419985e-04  1.22555822e-03  7.84839588e-05
   4.12724041e-04 -1.02130781e-03  1.29497293e-03  1.49869645e-04
   4.43089772e-04  4.88190727e-05 -2.92334821e-04 -7.37022362e-04
   1.31180640e-03  7.66336595e-04 -9.45436927e-04  3.25815559e-04
  -1.93969820e-04  2.95399112e-04 -1.31579044e-03 -7.92236008e-04
   7.66715765e-04  1.19458558e-04 -1.32288213e-03 -8.35896123e-04
  -2.73629179e-04 -2.16535452e-04 -1.11721015e-03  5.53301595e-04
   1.41583288e-03  1.97611173e-04 -4.60169196e-06  5.60535000e-05
  -9.84620813e-05 -8.83228263e-04 -5.31594330e-04  2.11475729e-03
   6.50282241e-04 -4.02156790e-04 -1.05456372e-03 -7.22117137e-05
  -8.02829965e-06  3.38121722e-04 -1.71762377e-04  2.33439376e-03
  -3.84842140e-04  2.41403671e-05  8.04463877e-04  1.93649476e-03
   5.58259036e-04  1.68970088e-04 -5.14430717e-04  4.16797958e-04
   4.73321

# Learning with the SGD optimizer

In [56]:
b_layer1 = copy.copy(layer1)
b_layer2 = copy.copy(layer2)
b_layer3 = copy.copy(layer3)
b_activation1 = copy.copy(activation1)
b_activation2 = copy.copy(activation2)
b_loss_activation = copy.copy(loss_activation)
b_acc = 0
for epoch in range(15001):
    layer1.forward(X)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    layer3.forward(activation2.output)
    loss = loss_activation.forward(layer3.output, Y)
    predictions = np.argmax(loss_activation.output,axis=1)
    accuracy = np.mean(predictions==Y)

    if epoch%500==0:
        print(f"Epoch: {epoch} | acc: {accuracy:.3f} | loss: {loss:.3f}")
    
    loss_activation.backward(loss_activation.output, Y)
    layer3.backward(loss_activation.dinputs)
    activation2.backward(layer3.dinputs)
    layer2.backward(activation2.dinputs)
    activation1.backward(layer2.dinputs)
    layer1.backward(activation1.dinputs)
    
    if accuracy>b_acc:
        b_layer1 = copy.copy(layer1)
        b_layer2 = copy.copy(layer2)
        b_layer3 = copy.copy(layer3)
        b_activation1 = copy.copy(activation1)
        b_activation2 = copy.copy(activation2)
        b_loss_activation = copy.copy(loss_activation)
        b_acc = accuracy
    
    optimizer.pre_update_params()
    optimizer.update_params(layer1)
    optimizer.update_params(layer2)
    optimizer.update_params(layer3)
    optimizer.post_update_params()

Epoch: 0 | acc: 0.449 | loss: 1.097
Epoch: 500 | acc: 0.725 | loss: 0.531
Epoch: 1000 | acc: 0.951 | loss: 0.139
Epoch: 1500 | acc: 0.955 | loss: 0.115
Epoch: 2000 | acc: 0.947 | loss: 0.120
Epoch: 2500 | acc: 0.962 | loss: 0.103
Epoch: 3000 | acc: 0.977 | loss: 0.077
Epoch: 3500 | acc: 0.962 | loss: 0.076
Epoch: 4000 | acc: 0.977 | loss: 0.055
Epoch: 4500 | acc: 0.977 | loss: 0.056
Epoch: 5000 | acc: 0.981 | loss: 0.064
Epoch: 5500 | acc: 0.977 | loss: 0.053
Epoch: 6000 | acc: 0.981 | loss: 0.037
Epoch: 6500 | acc: 0.992 | loss: 0.038
Epoch: 7000 | acc: 0.992 | loss: 0.028
Epoch: 7500 | acc: 0.996 | loss: 0.023
Epoch: 8000 | acc: 0.989 | loss: 0.023
Epoch: 8500 | acc: 0.996 | loss: 0.024
Epoch: 9000 | acc: 0.996 | loss: 0.016
Epoch: 9500 | acc: 0.989 | loss: 0.021
Epoch: 10000 | acc: 0.996 | loss: 0.015
Epoch: 10500 | acc: 1.000 | loss: 0.012
Epoch: 11000 | acc: 0.996 | loss: 0.012
Epoch: 11500 | acc: 1.000 | loss: 0.009
Epoch: 12000 | acc: 1.000 | loss: 0.008
Epoch: 12500 | acc: 1.00

In [57]:
X,Y = create_data(300,3)
b_layer1.forward(X)
b_activation1.forward(layer1.output)
b_layer2.forward(activation1.output)
b_activation2.forward(layer2.output)
b_layer3.forward(activation2.output)
loss = b_loss_activation.forward(b_layer3.output, Y)
predictions = np.argmax(b_loss_activation.output,axis=1)
accuracy = np.mean(predictions==Y)
print(f"Epoch: Validation | acc: {accuracy:.3f} | loss: {loss:.3f}")
correct = predictions == Y
incorrect = ~correct

plt.figure(figsize=(8, 8))

# Correct classifications (colored by class)
plt.scatter(
    X[correct, 0],
    X[correct, 1],
    c=Y[correct],
    cmap='brg',
    s=40,
    edgecolors='k',
    linewidths=0.3
)

# Incorrect classifications (black, semi-transparent)
plt.scatter(
    X[incorrect, 0],
    X[incorrect, 1],
    c='black',
    alpha=0.5,
    s=40
)

# Custom legend (fixes color mismatch)
legend_elements = [
    Line2D([0], [0], marker='o', color='w',
           markerfacecolor='white', markeredgecolor='black',
           markersize=8, label='Correct (colored by class)'),
    Line2D([0], [0], marker='o', color='w',
           markerfacecolor='black', alpha=0.5,
           markersize=8, label='Incorrect')
]

plt.legend(handles=legend_elements)

plt.title("Classification Results")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.axis('equal')
plt.savefig("./demo/spiralClassification.png", dpi=300, bbox_inches="tight")
plt.show()

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (265,) (793,) 